In [1]:
%load_ext autoreload
%autoreload 2

In [294]:
from ephesus.data import get_data_csv, get_data_json
from ephesus.sentence import return_label, load_model
from ephesus.duration import Duration
from ephesus.timedate import Date 
import numpy as np
import pandas as pd
from ephesus.duration import Duration
import re
import datetime

In [3]:
df = pd.DataFrame(get_data_csv()["translation"])

In [4]:
model = load_model("../models/model_full/model-best")

In [5]:
df

,translation
fichier,
9f980dcf-b431-4e67-876f-2b8e288b7900_3b5f3f01-e244-4398-8388-19fb5058c9c1,Prise de sang faite le 19 février à domicile p...
2d035c4b-cdfa-4982-87dc-916fe07a0824_710f4bda-ba21-434f-883e-1b72df7ea27a,Pansement également refait le 23 mars par mylène.
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_ff1697c1-a01f-4eb5-a1c2-b95205dc389a,Pcr plus prise de sang fait à domicile à 11.
3e7ffb67-8872-4172-bf18-4fe338ecae3f_6e8b49e5-4213-45da-bad6-158592c7f731,Prise de sang et analyse stérile des urines à ...
2d035c4b-cdfa-4982-87dc-916fe07a0824_56c749ad-eed8-407a-ae9e-a17ebf420cf4,"Bonjour, merci de renouveler cette ordonnance ..."
...,...
a5050600-95ef-43d8-a5fa-a57c791bf843_448853a4-f75e-403a-8fcb-8dfc18f0b4ec,Test antigénique réalisé cet après-midi à domi...
2d035c4b-cdfa-4982-87dc-916fe07a0824_4fb0c6a1-0562-496a-ae88-1adadec059a6,Réalisation d'une prise de sang chez monsieur ...
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_b8a5877a-b8ce-4347-b294-3aa46efbcb34,Ordonnance pour la préparation du pilulier en ...


In [6]:
df["entities"] = df["translation"].apply(lambda x: return_label(x, model))

In [7]:
def extract(prediction, label):
    for elm in prediction:
        if elm[1] == label:
            if elm[0] == np.nan:
                return np.nan
            else:
                return elm[0]  

In [8]:
df["raw_date"] = df["entities"].apply(lambda x: extract(x,'Date'))

In [9]:
df["Duration"] = df["entities"].apply(lambda x: extract(x,'Duration'))

In [10]:
def apply_date(sentence):
    if sentence == None:
        return None
    else: 
        df = Date().transform_data(sentence)
        return [df.iloc[0,2], df.iloc[0,1]]

In [11]:
test = 'le 10 juin'
temp = Date().transform_data(test)
temp

,data,day,month,year,day_of_week,day_from_today
0,"[10, juin]",10,6,99,99,99


In [12]:
apply_date(test)

[6, 10]

In [13]:
df["Date"] = df["raw_date"].apply(lambda x: apply_date(x))

In [14]:
df.drop(columns=["entities", "raw_date"], inplace=True)

In [15]:
df.head()

,translation,Duration,Date
fichier,,,
9f980dcf-b431-4e67-876f-2b8e288b7900_3b5f3f01-e244-4398-8388-19fb5058c9c1,Prise de sang faite le 19 février à domicile p...,None,"[2, 19]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_710f4bda-ba21-434f-883e-1b72df7ea27a,Pansement également refait le 23 mars par mylène.,None,"[3, 23]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_ff1697c1-a01f-4eb5-a1c2-b95205dc389a,Pcr plus prise de sang fait à domicile à 11.,None,None
3e7ffb67-8872-4172-bf18-4fe338ecae3f_6e8b49e5-4213-45da-bad6-158592c7f731,Prise de sang et analyse stérile des urines à ...,None,"[2, 18]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_56c749ad-eed8-407a-ae9e-a17ebf420cf4,"Bonjour, merci de renouveler cette ordonnance ...",None,None


In [16]:
Duration().explicit_duration("pendant 3 semaines")

{'CareDuration': ['3'], 'CareDurationType': ['Weeks'], 'CareEnd': [nan]}

In [17]:
test = Duration()

In [18]:
test.explicit_duration("pendant 3 jours")

{'CareDuration': ['3'], 'CareDurationType': ['Days'], 'CareEnd': [nan]}

In [19]:
df.iloc[0,2][0] = df.iloc[0,2][0] + 2 

In [20]:
df.iloc[0,2][0]

4

In [21]:
test.liste_mois

'(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)'

In [22]:
test.implicit_duration_1('jusqu\'au 24 juin')

{'CareDuration': [nan], 'CareDurationType': [nan], 'CareEnd': ['24 juin']}

In [23]:
test.implicit_duration_1('jusqu\'au 24 juin',(6,10))

{'CareDuration': [14], 'CareDurationType': ['Days'], 'CareEnd': ['24 juin']}

In [24]:
test.get_duration(sentence='jusqu\'au 16 juin', CareBeginDate=[6,20])

{'CareDuration': [nan], 'CareDurationType': ['Days'], 'CareEnd': ['16 juin']}

In [25]:
liste_mois = '(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)'

In [26]:
re.search("(?<=jusqu\'au\s)(\w\s)*\d{1,2}\s"+liste_mois, 'jusqu\'au 16 juin')

<re.Match object; span=(9, 16), match='16 juin'>

In [27]:
test.get_duration(sentence="jusqu'au 16 juin")

{'CareDuration': [nan], 'CareDurationType': [nan], 'CareEnd': ['16 juin']}

In [28]:
test.get_duration(sentence='pour 10 jours')

{'CareDuration': ['10'], 'CareDurationType': ['Days'], 'CareEnd': [nan]}

In [29]:
def get_duration_df(x):
    if x is None:
        return None

In [30]:
test = pd.DataFrame()

In [31]:
type(test) == pd.core.frame.DataFrame

True

In [32]:
type(df["Duration"])

pandas.core.series.Series

In [33]:
duration = df["Duration"]
date = df["Date"]

In [34]:
df_temp = df.dropna()
df_temp

,translation,Duration,Date
fichier,,,
55d674cc-3389-4cf6-ab7c-1f1b9fa1b6ed_3e005951-47a1-4edb-ab50-ce602fe56c54,Renouvellement du bsc. D'une durée de un an à ...,une durée de un an,"[5, 21]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_2dfdf4ab-fc95-4ec1-8e03-af502b95baf8,Ordonnance pour la réalisation de injection de...,pour 6 jours,"[4, 17]"
c619f0e1-7c22-4047-adb2-df4dec6643ba_8214d80f-0f7d-4e0d-9d4a-23294f9690b3,Soin débuté le vendredi 14 janvier 2022 jusqu'...,jusqu'au lundi 17 janvier 2022,"[1, 14]"
f0caa21b-c11d-40a3-98ec-e2df3d2b9cc4_7e5c146c-2faa-44ce-9228-ee5addf48580,"Soin débuté le 10 avril, à réaliser donc le s...",pendant 15 jours,"[4, 10]"
9d42ce6f-8537-49a9-a121-c8ae1dc7cda8_3ffbf9cc-b9e4-405d-a038-fb4f9e036a72,"Pour monsieur Didier Nicolas, donc un passage ...",jusqu'au 8/12/2019,"[99, 25]"
...,...,...,...
3e7ffb67-8872-4172-bf18-4fe338ecae3f_04fe104b-9843-4abc-b467-a9301c5eb3a2,Injection sous-cutanée à réaliser une fois par...,pendant 8 jours,"[7, 9]"
3e7ffb67-8872-4172-bf18-4fe338ecae3f_67dbeda4-1876-41a6-8249-68f1dfeebb70,Nouvelle ordonnance pour madame Gris Rennes. O...,pendant un mois,"[7, 20]"
3e7ffb67-8872-4172-bf18-4fe338ecae3f_80f9b3d2-47b8-4152-9e45-a26dcf173335,Injection sous-cutanée à faire une fois par jo...,pendant 30 jours,"[4, 7]"


In [35]:
test_1 = Duration().explicit_duration("pour 4 jours")

In [36]:
test_2 = Duration().explicit_duration("pendant 8 mois")

In [37]:
full_test = pd.DataFrame({'test':[test_1,test_2]})
full_test

,test
0,"{'CareDuration': ['4'], 'CareDurationType': ['..."
1,"{'CareDuration': ['8'], 'CareDurationType': ['..."


In [38]:
full_test['CareDuration'] = full_test["test"].apply(lambda x:x['CareDuration'][0])

In [39]:
full_test

,test,CareDuration
0,"{'CareDuration': ['4'], 'CareDurationType': ['...",4
1,"{'CareDuration': ['8'], 'CareDurationType': ['...",8


In [40]:
Duration().implicit_duration_1("jusqu'au 20 juin", [6,10],df=True)

,CareDuration,CareDurationType,CareEnd
0,10,Days,20 juin


In [41]:
df

,translation,Duration,Date
fichier,,,
9f980dcf-b431-4e67-876f-2b8e288b7900_3b5f3f01-e244-4398-8388-19fb5058c9c1,Prise de sang faite le 19 février à domicile p...,None,"[4, 19]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_710f4bda-ba21-434f-883e-1b72df7ea27a,Pansement également refait le 23 mars par mylène.,None,"[3, 23]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_ff1697c1-a01f-4eb5-a1c2-b95205dc389a,Pcr plus prise de sang fait à domicile à 11.,None,None
3e7ffb67-8872-4172-bf18-4fe338ecae3f_6e8b49e5-4213-45da-bad6-158592c7f731,Prise de sang et analyse stérile des urines à ...,None,"[2, 18]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_56c749ad-eed8-407a-ae9e-a17ebf420cf4,"Bonjour, merci de renouveler cette ordonnance ...",None,None
...,...,...,...
a5050600-95ef-43d8-a5fa-a57c791bf843_448853a4-f75e-403a-8fcb-8dfc18f0b4ec,Test antigénique réalisé cet après-midi à domi...,None,"[99, 99]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_4fb0c6a1-0562-496a-ae88-1adadec059a6,Réalisation d'une prise de sang chez monsieur ...,None,"[2, 24]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_b8a5877a-b8ce-4347-b294-3aa46efbcb34,Ordonnance pour la préparation du pilulier en ...,None,"[2, 1]"


In [42]:
def test(x):
    if x is None:
        return None
    return Duration().implicit_duration_1(x)

In [59]:
df_temp = df.copy()

In [60]:
df_temp["test"] = df_temp["Duration"].apply(lambda x: test(x))

TypeError: 'str' object is not callable

In [61]:
df_temp.isna().sum()

translation       0
Duration       3655
Date            652
dtype: int64

In [46]:
test = "jusqu'au 6/6/2022"
df_blabla = Date().transform_data(test)
df_blabla

,data,day,month,year,day_of_week,day_from_today
0,"[jusqu, 6, 6, 2022]",6,6,2022,99,99


In [47]:
Duration().implicit_duration_1(test, [6,2], df=True)

,CareDuration,CareDurationType,CareEnd
0,4,Days,6 juin


In [48]:
test = "jusqu'au 10 juin"

In [49]:
Duration().get_duration(test, [6,99], df=True)

,CareDuration,CareDurationType,CareEnd
0,NaN,NaN,10 juin


In [50]:
df

,translation,Duration,Date
fichier,,,
9f980dcf-b431-4e67-876f-2b8e288b7900_3b5f3f01-e244-4398-8388-19fb5058c9c1,Prise de sang faite le 19 février à domicile p...,None,"[4, 19]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_710f4bda-ba21-434f-883e-1b72df7ea27a,Pansement également refait le 23 mars par mylène.,None,"[3, 23]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_ff1697c1-a01f-4eb5-a1c2-b95205dc389a,Pcr plus prise de sang fait à domicile à 11.,None,None
3e7ffb67-8872-4172-bf18-4fe338ecae3f_6e8b49e5-4213-45da-bad6-158592c7f731,Prise de sang et analyse stérile des urines à ...,None,"[2, 18]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_56c749ad-eed8-407a-ae9e-a17ebf420cf4,"Bonjour, merci de renouveler cette ordonnance ...",None,None
...,...,...,...
a5050600-95ef-43d8-a5fa-a57c791bf843_448853a4-f75e-403a-8fcb-8dfc18f0b4ec,Test antigénique réalisé cet après-midi à domi...,None,"[99, 99]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_4fb0c6a1-0562-496a-ae88-1adadec059a6,Réalisation d'une prise de sang chez monsieur ...,None,"[2, 24]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_b8a5877a-b8ce-4347-b294-3aa46efbcb34,Ordonnance pour la préparation du pilulier en ...,None,"[2, 1]"


In [102]:
df_temp = df.copy()

In [103]:
df_temp.isna().sum()

translation       0
Duration       3655
Date            652
dtype: int64

In [101]:
print(Duration().get_duration('ma bite'))

{'CareDuration': [nan], 'CareDurationType': [nan], 'CareEnd': [nan]}


In [104]:
test = Duration().apply_to_df(df_temp)
test

,translation,Duration,Date,CareDuration,CareDurationType,CareEnd
fichier,,,,,,
9f980dcf-b431-4e67-876f-2b8e288b7900_3b5f3f01-e244-4398-8388-19fb5058c9c1,Prise de sang faite le 19 février à domicile p...,None,"[4, 19]",NaN,NaN,NaN
2d035c4b-cdfa-4982-87dc-916fe07a0824_710f4bda-ba21-434f-883e-1b72df7ea27a,Pansement également refait le 23 mars par mylène.,None,"[3, 23]",NaN,NaN,NaN
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_ff1697c1-a01f-4eb5-a1c2-b95205dc389a,Pcr plus prise de sang fait à domicile à 11.,None,None,NaN,NaN,NaN
3e7ffb67-8872-4172-bf18-4fe338ecae3f_6e8b49e5-4213-45da-bad6-158592c7f731,Prise de sang et analyse stérile des urines à ...,None,"[2, 18]",NaN,NaN,NaN
2d035c4b-cdfa-4982-87dc-916fe07a0824_56c749ad-eed8-407a-ae9e-a17ebf420cf4,"Bonjour, merci de renouveler cette ordonnance ...",None,None,NaN,NaN,NaN
...,...,...,...,...,...,...
a5050600-95ef-43d8-a5fa-a57c791bf843_448853a4-f75e-403a-8fcb-8dfc18f0b4ec,Test antigénique réalisé cet après-midi à domi...,None,"[99, 99]",NaN,NaN,NaN
2d035c4b-cdfa-4982-87dc-916fe07a0824_4fb0c6a1-0562-496a-ae88-1adadec059a6,Réalisation d'une prise de sang chez monsieur ...,None,"[2, 24]",NaN,NaN,NaN
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_b8a5877a-b8ce-4347-b294-3aa46efbcb34,Ordonnance pour la préparation du pilulier en ...,None,"[2, 1]",NaN,NaN,NaN


In [86]:
test.isna().sum()

translation           0
Duration              0
Date                  0
CareDuration         78
CareDurationType     75
CareEnd             287
dtype: int64

In [87]:
test

,translation,Duration,Date,CareDuration,CareDurationType,CareEnd
fichier,,,,,,
55d674cc-3389-4cf6-ab7c-1f1b9fa1b6ed_3e005951-47a1-4edb-ab50-ce602fe56c54,Renouvellement du bsc. D'une durée de un an à ...,une durée de un an,"[5, 21]",1,Year,NaN
2d035c4b-cdfa-4982-87dc-916fe07a0824_2dfdf4ab-fc95-4ec1-8e03-af502b95baf8,Ordonnance pour la réalisation de injection de...,pour 6 jours,"[4, 17]",6,Days,NaN
c619f0e1-7c22-4047-adb2-df4dec6643ba_8214d80f-0f7d-4e0d-9d4a-23294f9690b3,Soin débuté le vendredi 14 janvier 2022 jusqu'...,jusqu'au lundi 17 janvier 2022,"[1, 14]",3,Days,17 janvier
f0caa21b-c11d-40a3-98ec-e2df3d2b9cc4_7e5c146c-2faa-44ce-9228-ee5addf48580,"Soin débuté le 10 avril, à réaliser donc le s...",pendant 15 jours,"[4, 10]",15,Days,NaN
9d42ce6f-8537-49a9-a121-c8ae1dc7cda8_3ffbf9cc-b9e4-405d-a038-fb4f9e036a72,"Pour monsieur Didier Nicolas, donc un passage ...",jusqu'au 8/12/2019,"[99, 25]",NaN,NaN,8 décembre
...,...,...,...,...,...,...
3e7ffb67-8872-4172-bf18-4fe338ecae3f_04fe104b-9843-4abc-b467-a9301c5eb3a2,Injection sous-cutanée à réaliser une fois par...,pendant 8 jours,"[7, 9]",8,Days,NaN
3e7ffb67-8872-4172-bf18-4fe338ecae3f_67dbeda4-1876-41a6-8249-68f1dfeebb70,Nouvelle ordonnance pour madame Gris Rennes. O...,pendant un mois,"[7, 20]",1,Months,NaN
3e7ffb67-8872-4172-bf18-4fe338ecae3f_80f9b3d2-47b8-4152-9e45-a26dcf173335,Injection sous-cutanée à faire une fois par jo...,pendant 30 jours,"[4, 7]",30,Days,NaN


In [54]:
df_temp

,translation,Duration,Date,temp
fichier,,,,
55d674cc-3389-4cf6-ab7c-1f1b9fa1b6ed_3e005951-47a1-4edb-ab50-ce602fe56c54,Renouvellement du bsc. D'une durée de un an à ...,une durée de un an,"[5, 21]","{'CareDuration': ['1'], 'CareDurationType': ['..."
2d035c4b-cdfa-4982-87dc-916fe07a0824_2dfdf4ab-fc95-4ec1-8e03-af502b95baf8,Ordonnance pour la réalisation de injection de...,pour 6 jours,"[4, 17]","{'CareDuration': ['6'], 'CareDurationType': ['..."
c619f0e1-7c22-4047-adb2-df4dec6643ba_8214d80f-0f7d-4e0d-9d4a-23294f9690b3,Soin débuté le vendredi 14 janvier 2022 jusqu'...,jusqu'au lundi 17 janvier 2022,"[1, 14]","{'CareDuration': [3], 'CareDurationType': ['Da..."
f0caa21b-c11d-40a3-98ec-e2df3d2b9cc4_7e5c146c-2faa-44ce-9228-ee5addf48580,"Soin débuté le 10 avril, à réaliser donc le s...",pendant 15 jours,"[4, 10]","{'CareDuration': ['15'], 'CareDurationType': [..."
9d42ce6f-8537-49a9-a121-c8ae1dc7cda8_3ffbf9cc-b9e4-405d-a038-fb4f9e036a72,"Pour monsieur Didier Nicolas, donc un passage ...",jusqu'au 8/12/2019,"[99, 25]","{'CareDuration': [nan], 'CareDurationType': [n..."
...,...,...,...,...
2d035c4b-cdfa-4982-87dc-916fe07a0824_558bf9c3-a9e5-408e-bc55-da9844e1b61a,Ordonnance de renouvellement pour monsieur Did...,pour 3 mois,"[6, 29]","{'CareDuration': ['3'], 'CareDurationType': ['..."
3e7ffb67-8872-4172-bf18-4fe338ecae3f_04fe104b-9843-4abc-b467-a9301c5eb3a2,Injection sous-cutanée à réaliser une fois par...,pendant 8 jours,"[7, 9]","{'CareDuration': ['8'], 'CareDurationType': ['..."
3e7ffb67-8872-4172-bf18-4fe338ecae3f_67dbeda4-1876-41a6-8249-68f1dfeebb70,Nouvelle ordonnance pour madame Gris Rennes. O...,pendant un mois,"[7, 20]","{'CareDuration': ['1'], 'CareDurationType': ['..."


In [55]:
for i in range(100):
    sentence = df_temp.iloc[i,0]
    if re.search("jusqu\'au\s", sentence) is not None:
        print(df_temp.iloc[i,1])
        print(df_temp.iloc[i,2])
        print(df_temp.iloc[i,3])

pour 6 jours
[4, 17]
{'CareDuration': ['6'], 'CareDurationType': ['Days'], 'CareEnd': [nan]}
jusqu'au lundi 17 janvier 2022
[1, 14]
{'CareDuration': [3], 'CareDurationType': ['Days'], 'CareEnd': ['17 janvier']}
jusqu'au 8/12/2019
[99, 25]
{'CareDuration': [nan], 'CareDurationType': [nan], 'CareEnd': ['8 décembre']}
jusqu'au 28 novembre
[11, 23]
{'CareDuration': [5], 'CareDurationType': ['Days'], 'CareEnd': ['28 novembre']}
3 mois
[12, 11]
{'CareDuration': ['3'], 'CareDurationType': ['Months'], 'CareEnd': [nan]}
jusqu'au dimanche 7
[3, 4]
{'CareDuration': [nan], 'CareDurationType': [nan], 'CareEnd': [nan]}
jusqu'au 20 février
[2, 12]
{'CareDuration': [8], 'CareDurationType': ['Days'], 'CareEnd': ['20 février']}
jusqu'au 4 mars
[2, 21]
{'CareDuration': [11], 'CareDurationType': ['Days'], 'CareEnd': ['4 mars']}
jusqu'au 27 octobre
[9, 21]
{'CareDuration': [36], 'CareDurationType': ['Days'], 'CareEnd': ['27 octobre']}
jusqu'au 17 janvier
[1, 7]
{'CareDuration': [10], 'CareDurationType': ['

In [56]:
df_bb = Duration("../models/model_full/model-best").get_data()

In [57]:
df_bb

,translation,Duration,Date
fichier,,,
9f980dcf-b431-4e67-876f-2b8e288b7900_3b5f3f01-e244-4398-8388-19fb5058c9c1,Prise de sang faite le 19 février à domicile p...,None,"[2, 19]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_710f4bda-ba21-434f-883e-1b72df7ea27a,Pansement également refait le 23 mars par mylène.,None,"[3, 23]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_ff1697c1-a01f-4eb5-a1c2-b95205dc389a,Pcr plus prise de sang fait à domicile à 11.,None,None
3e7ffb67-8872-4172-bf18-4fe338ecae3f_6e8b49e5-4213-45da-bad6-158592c7f731,Prise de sang et analyse stérile des urines à ...,None,"[2, 18]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_56c749ad-eed8-407a-ae9e-a17ebf420cf4,"Bonjour, merci de renouveler cette ordonnance ...",None,None
...,...,...,...
a5050600-95ef-43d8-a5fa-a57c791bf843_448853a4-f75e-403a-8fcb-8dfc18f0b4ec,Test antigénique réalisé cet après-midi à domi...,None,"[99, 99]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_4fb0c6a1-0562-496a-ae88-1adadec059a6,Réalisation d'une prise de sang chez monsieur ...,None,"[2, 24]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_b8a5877a-b8ce-4347-b294-3aa46efbcb34,Ordonnance pour la préparation du pilulier en ...,None,"[2, 1]"


### Test for du XX au XX

In [126]:
liste_mois = '(janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre)'

In [159]:
test = "du lundi 17 juin au 22 juin 2022"
re.search("du\s\w*?\s?(\d{1,2}\sjuin\s)(\d{1,4}\s)?au\s\w*?\s?(\d{1,2}\sjuin)", test)

<re.Match object; span=(0, 27), match='du lundi 17 juin au 22 juin'>

In [146]:
regex = "du\s\w*?\s?(\d{1,2}\s"+liste_mois+"\s)(\d{1,4}\s)?au\s\w*?\s?(\d{1,2}\s"+liste_mois+")"
x = re.search(regex, test)

In [147]:
x.group()

'du 17 juin au 22 juin'

In [160]:
Duration().implicit_duration_2(test, df=True)

,CareDuration,CareDurationType,CareEnd
0,5,Days,22 juin


In [148]:
end_date = re.search('(?<=au\s)\w*?\s?(\d{1,2}\s'+liste_mois+')', test)

In [152]:
end_date.group()

'22 juin'

In [161]:
Duration().get_duration(test, df=True)

,CareDuration,CareDurationType,CareEnd
0,5,Days,22 juin


In [164]:
last_test = Duration(path_spacy = "../models/model_full/model-best").get_data()

In [166]:
last_test = Duration().apply_to_df(last_test)

In [181]:
last_test_copy = last_test.copy().dropna(subset=["Duration","CareEnd"])
last_test_copy

,translation,Duration,Date,CareDuration,CareDurationType,CareEnd
fichier,,,,,,
c619f0e1-7c22-4047-adb2-df4dec6643ba_8214d80f-0f7d-4e0d-9d4a-23294f9690b3,Soin débuté le vendredi 14 janvier 2022 jusqu'...,jusqu'au lundi 17 janvier 2022,"[1, 14]",3,Days,17 janvier
9d42ce6f-8537-49a9-a121-c8ae1dc7cda8_3ffbf9cc-b9e4-405d-a038-fb4f9e036a72,"Pour monsieur Didier Nicolas, donc un passage ...",jusqu'au 8/12/2019,"[99, 25]",NaN,NaN,8 décembre
9d42ce6f-8537-49a9-a121-c8ae1dc7cda8_273dbe94-8234-4a98-9f7f-43ae33995686,"Pour madame Didier Nicole, un passage à domici...",jusqu'au 28 novembre,"[11, 23]",5,Days,28 novembre
a5050600-95ef-43d8-a5fa-a57c791bf843_7e6b8930-4733-4700-a0ac-40d28720c7ec,Injection d'anticoagulants à planifier tous le...,du 5 mars au 14 mars,None,9,Days,14 mars
55d674cc-3389-4cf6-ab7c-1f1b9fa1b6ed_4915ce53-0693-b8fe-67bf-cda0c15a57fe,C'est un forfait bsc matin et soir et c'est ju...,jusqu'au 16 avril,None,NaN,NaN,16 avril
3e7ffb67-8872-4172-bf18-4fe338ecae3f_98c34931-c7d8-4eb2-9ef3-fac00eaad6a1,Ordonnance pour la réalisation à domicile par ...,jusqu'au 20 février,"[2, 12]",8,Days,20 février
74b154c1-e265-4cb9-8e71-0c9bbc3dc880_0a7fd90b-e718-4cca-bc3b-ee47a3b89b31,"Petite précision pour monsieur jean-paul, il e...",jusqu'au 18 juillet,None,NaN,NaN,18 juillet
9d42ce6f-8537-49a9-a121-c8ae1dc7cda8_9e347870-1b98-479e-ad0b-8e2153695491,"Pansement à domicile, juin le 21 février. C'es...",jusqu'au 4 mars,"[2, 21]",11,Days,4 mars
978eb126-14a0-46cf-bd27-44b6b77cecb7_3b1ee428-2b8b-41c7-a04c-bebe224b2353,Ordonnance de renouvellement pour les sous-cut...,jusqu'au 27 octobre,"[9, 21]",36,Days,27 octobre


In [182]:
last_test_v2 = Duration(path_spacy = "../models/model_full/model-best").get_data()
last_test_v2

,translation,Duration,Date
fichier,,,
9f980dcf-b431-4e67-876f-2b8e288b7900_3b5f3f01-e244-4398-8388-19fb5058c9c1,Prise de sang faite le 19 février à domicile p...,None,"[2, 19]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_710f4bda-ba21-434f-883e-1b72df7ea27a,Pansement également refait le 23 mars par mylène.,None,"[3, 23]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_ff1697c1-a01f-4eb5-a1c2-b95205dc389a,Pcr plus prise de sang fait à domicile à 11.,None,None
3e7ffb67-8872-4172-bf18-4fe338ecae3f_6e8b49e5-4213-45da-bad6-158592c7f731,Prise de sang et analyse stérile des urines à ...,None,"[2, 18]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_56c749ad-eed8-407a-ae9e-a17ebf420cf4,"Bonjour, merci de renouveler cette ordonnance ...",None,None
...,...,...,...
a5050600-95ef-43d8-a5fa-a57c791bf843_448853a4-f75e-403a-8fcb-8dfc18f0b4ec,Test antigénique réalisé cet après-midi à domi...,None,"[99, 99]"
2d035c4b-cdfa-4982-87dc-916fe07a0824_4fb0c6a1-0562-496a-ae88-1adadec059a6,Réalisation d'une prise de sang chez monsieur ...,None,"[2, 24]"
b96ad235-2f4f-4236-9aa3-9df85dfcf7cb_b8a5877a-b8ce-4347-b294-3aa46efbcb34,Ordonnance pour la préparation du pilulier en ...,None,"[2, 1]"


In [316]:
copycopy = last_test_v2.copy().dropna(subset=["Duration"])
print(copycopy.shape)

(374, 3)


In [317]:
copycopy['test'] = copycopy["Duration"].apply(lambda x: re.search("(?<=du\s)\w*?\s?(\d{1,2}\s)",x).string if re.search("(?<=du\s)\w*?\s?(\d{1,2}\s)",x) is not None else None)

In [323]:
extract_test = copycopy.dropna(subset=["test"])
extract_test

,translation,Duration,Date,test
fichier,,,,
a5050600-95ef-43d8-a5fa-a57c791bf843_7e6b8930-4733-4700-a0ac-40d28720c7ec,Injection d'anticoagulants à planifier tous le...,du 5 mars au 14 mars,None,du 5 mars au 14 mars
9d42ce6f-8537-49a9-a121-c8ae1dc7cda8_3492f087-bb2b-49ae-82d5-2da4fa3656b6,Administration traitement ehpad Claudette Chen...,du 25 au 28 février,None,du 25 au 28 février
55d674cc-3389-4cf6-ab7c-1f1b9fa1b6ed_94d30710-8eb0-4bbb-8bea-f0d577006f5d,Faire pratiquer par une infirmière à domicile ...,du 12 au 21 février,"[2, 21]",du 12 au 21 février
3e7ffb67-8872-4172-bf18-4fe338ecae3f_18f3b4fe-e953-471e-884d-2e7cede92edb,Merci de planifier également des prises de san...,du 14 mars,"[3, 14]",du 14 mars
74b154c1-e265-4cb9-8e71-0c9bbc3dc880_6a8c906a-1c6b-4bc6-b86a-d21a553c6ed4,"monsieur Didier Nicolas, ordonnance du 25 mars...",du 25 mars,"[3, 25]",du 25 mars
9d42ce6f-8537-49a9-a121-c8ae1dc7cda8_47fa45f9-0a6f-4709-9184-3e58865cb49d,Administration traitement ehpad Claudette Chen...,du 25 au 28 février,None,du 25 au 28 février
9f980dcf-b431-4e67-876f-2b8e288b7900_1d110f73-529d-474c-a36a-4f6f3a713d6b,Intramusculaire programmé matin et soir. parti...,du 3 mars et jusqu'au 6 mars inclus,None,du 3 mars et jusqu'au 6 mars inclus
2206f47b-a858-4f23-8696-d10e1050f2d7_2a6c1cf5-5afa-4970-948a-7c46571a26fc,Donc nouvelle ordonnance par madame Didier Den...,du 11 décembre,"[12, 12]",du 11 décembre
8a3df958-343b-4cf2-9134-6c49839ebf08_2239280f-fed7-4cbe-b172-aa97a7ff48fb,"Alors madame fabre, on commencé du 25 et on fi...",du 25 et on finit,"[99, 99]",du 25 et on finit


In [324]:
extract_test = Duration().apply_to_df(extract_test)

In [325]:
extract_test.shape

(12, 7)

In [326]:
extract_test

,translation,Duration,Date,test,CareDuration,CareDurationType,CareEnd
fichier,,,,,,,
a5050600-95ef-43d8-a5fa-a57c791bf843_7e6b8930-4733-4700-a0ac-40d28720c7ec,Injection d'anticoagulants à planifier tous le...,du 5 mars au 14 mars,None,du 5 mars au 14 mars,9.0,Days,14 mars
9d42ce6f-8537-49a9-a121-c8ae1dc7cda8_3492f087-bb2b-49ae-82d5-2da4fa3656b6,Administration traitement ehpad Claudette Chen...,du 25 au 28 février,None,du 25 au 28 février,3.0,Days,28 février
55d674cc-3389-4cf6-ab7c-1f1b9fa1b6ed_94d30710-8eb0-4bbb-8bea-f0d577006f5d,Faire pratiquer par une infirmière à domicile ...,du 12 au 21 février,"[2, 21]",du 12 au 21 février,9.0,Days,21 février
3e7ffb67-8872-4172-bf18-4fe338ecae3f_18f3b4fe-e953-471e-884d-2e7cede92edb,Merci de planifier également des prises de san...,du 14 mars,"[3, 14]",du 14 mars,NaN,NaN,NaN
74b154c1-e265-4cb9-8e71-0c9bbc3dc880_6a8c906a-1c6b-4bc6-b86a-d21a553c6ed4,"monsieur Didier Nicolas, ordonnance du 25 mars...",du 25 mars,"[3, 25]",du 25 mars,NaN,NaN,NaN
9d42ce6f-8537-49a9-a121-c8ae1dc7cda8_47fa45f9-0a6f-4709-9184-3e58865cb49d,Administration traitement ehpad Claudette Chen...,du 25 au 28 février,None,du 25 au 28 février,3.0,Days,28 février
9f980dcf-b431-4e67-876f-2b8e288b7900_1d110f73-529d-474c-a36a-4f6f3a713d6b,Intramusculaire programmé matin et soir. parti...,du 3 mars et jusqu'au 6 mars inclus,None,du 3 mars et jusqu'au 6 mars inclus,NaN,NaN,3 mars
2206f47b-a858-4f23-8696-d10e1050f2d7_2a6c1cf5-5afa-4970-948a-7c46571a26fc,Donc nouvelle ordonnance par madame Didier Den...,du 11 décembre,"[12, 12]",du 11 décembre,NaN,NaN,NaN
8a3df958-343b-4cf2-9134-6c49839ebf08_2239280f-fed7-4cbe-b172-aa97a7ff48fb,"Alors madame fabre, on commencé du 25 et on fi...",du 25 et on finit,"[99, 99]",du 25 et on finit,NaN,NaN,NaN


In [286]:
sentence = "du 23 au 27 juin"

In [276]:
regex = "du\s\w*?\s?(\d{1,2})\s("+liste_mois+"\s)?(\d{1,4}\s)?au\s\w*?\s?(\d{1,2}\s"+liste_mois+")"
re.search(regex, test)

<re.Match object; span=(0, 32), match='du lundi 23 juin 2022 au 27 juin'>

In [277]:
regex = "(?<=du\s)\w*?\s?(\d{1,2})\s("+liste_mois+")?\s?"
re.search(regex, test)

<re.Match object; span=(3, 17), match='lundi 23 juin '>

In [284]:
regex = "(?<=du\s)\w*?\s?(\d{1,2})\s("+liste_mois+")?\s?(\d{1,4}\s)?au\s\w*?\s?(\d{1,2}\s"+liste_mois+")"
re.search(regex, test)

<re.Match object; span=(3, 16), match='23 au 27 juin'>

In [304]:
test = re.search(regex, sentence)
print(test)
dico = {}
start_date = re.search("(?<=du\s)\w*?\s?(\d{1,2})\s("+liste_mois+")?\s?", sentence).group()
end_date = re.search('(?<=au\s)\w*?\s?(\d{1,2}\s'+liste_mois+')', sentence).group()
print(start_date)
print(end_date)

<re.Match object; span=(3, 16), match='23 au 27 juin'>
23 
27 juin


In [305]:
start_date_df = Date().transform_data(start_date)
end_date_df = Date().transform_data(end_date)

In [306]:
start_date_df

,data,day,month,year,day_of_week,day_from_today
0,[23],23,99,99,99,99


In [307]:
end_date_df

,data,day,month,year,day_of_week,day_from_today
0,"[27, juin]",27,6,99,99,99


In [308]:
end_date_datetime = datetime.date(2022,end_date_df.iloc[0,2],end_date_df.iloc[0,1])
end_date_datetime

datetime.date(2022, 6, 27)

In [309]:
if start_date_df.iloc[0,2] == 99:
    start_month = end_date_df.iloc[0,2]
else:
    start_month = start_date_df.iloc[0,2]
start_date_datetime = datetime.date(2022,start_month,start_date_df.iloc[0,1])
start_date_datetime

datetime.date(2022, 6, 23)

In [310]:
diff = (end_date_datetime - start_date_datetime).days
diff

4

In [311]:
if diff <= 0:
    dico["CareDuration"] = [np.nan]
else:
    dico["CareDuration"] = [diff]
        # by default, CareDurationType is Days
    dico["CareDurationType"] = ["Days"]
    dico["CareEnd"] = [end_date]

In [312]:
dico

{'CareDuration': [4], 'CareDurationType': ['Days'], 'CareEnd': ['27 juin']}

In [313]:
Duration().implicit_duration_2(sentence)

{'CareDuration': [4], 'CareDurationType': ['Days'], 'CareEnd': ['27 juin']}

In [328]:
sentence = "du lundi 23 juin 2022 et jusqu'au 27 juin"
Duration().get_duration(sentence)

{'CareDuration': [nan], 'CareDurationType': [nan], 'CareEnd': ['27 juin']}